In [2]:
from dotenv import load_dotenv
from openai import OpenAI
import os, threading, jsonlines, json

load_dotenv()

client = OpenAI(
    api_key=os.environ.get("openAIKey"),
)

In [3]:
expressions=dict()
threads=[]

In [4]:
def convert_sentence(i,sentence):
    response = client.chat.completions.create(
    messages=[
        {
            "role":"system",
            "content":"Convert given sentence to simple expression"
        },
        {
            "role": "user",
            "content": sentence,
        }
    ],
    model=os.environ.get("phase2Model"),
    )
    expressions[f"{i}"]=str(response.choices[0].message.content)

In [5]:
def get_expressions(sentences):
    for i,sentence in enumerate(sentences):
        t=threading.Thread(target=convert_sentence,args=(i,sentence,))
        threads.append(t)
        t.start()
    for t in threads:
        t.join()
    output=[]
    for i in range(len(sentences)):
        output.append(expressions[f"{i}"])
    return output
    

In [8]:
def getSentences():
    sentences=[]
    with jsonlines.open(f'../sample_data/{os.environ.get("testKey")}.jsonl') as data:
        for obj in data:
            if obj['type']=='sentence':
                sentences.append(obj['value'])
    return sentences

In [9]:
sentences = getSentences()
with open(f"../sample_data/{os.environ.get('expressionsKey')}.json", "w") as fp:
    json.dump(get_expressions(sentences), fp)